In [1]:
import os
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn import model_selection
import tensorflow as tf
from __future__ import print_function
from tensorflow.python.ops import resources
from tensorflow.contrib.tensor_forest.python import tensor_forest

folder=r'D:\Ameyem Office\Geoservices\Neeraj & me\log data\\'
df=pd.read_csv('lasdata.csv')

In [7]:
# las.data.transpose()
Xd=df[df.columns[[0,1,2,3,4,7,9]]]
Yd=df[df.columns[[0,5,6,8,10]]]

X1 = Xd.values[:,0:4] 
Y1 = Yd.values[:,1:2] 
validation_size = 0.20
seed = 7
Xo_train, Xo_test, Y_train, Y_test = model_selection.train_test_split(X1, Y1, test_size=validation_size, random_state=seed)
X_train, X_test =Xo_train[:,1:], Xo_test[:,1:]
scoring = 'accuracy'

n_samples,num_features=X_train.shape
num_classes=Y_train.shape[1]
batch_size=n_samples
# Parameters
batch_size,num_features,num_classes


(1124, 3, 1)

In [8]:
X_train,Y_train

(array([[126.1824,  59.2583,   8.3752],
        [119.6835,  58.3405,   5.0548],
        [126.3542,  65.4497,   5.1967],
        ...,
        [107.9646,  55.1028,   8.0347],
        [101.4942,  53.6725,   5.374 ],
        [116.2145,  64.7008,   5.9566]]), array([[1.00000e-04],
        [9.78090e-02],
        [1.16712e-01],
        ...,
        [5.64620e-02],
        [1.00000e-04],
        [6.89330e-02]]))

In [9]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
# Fit only to the training data
scaler_x.fit(X_train)
# Now apply the transformations to the data:
X_train = scaler_x.transform(X_train)
X_test= scaler_x.transform(X_validation)

scaler_y = StandardScaler()
# Fit only to the training data
scaler_y.fit(Y_train)
# Now apply the transformations to the data:
Y_train = scaler_y.transform(Y_train)
Y_test = scaler_y.transform(Y_test)
X_train,Y_train

(array([[ 1.17504613, -0.33069632,  0.47008064],
        [ 0.45171051, -0.46290355, -0.59352036],
        [ 1.19416769,  0.56116234, -0.5480665 ],
        ...,
        [-0.85261758, -0.92928769,  0.36101061],
        [-1.57278111, -1.13531952, -0.49127321],
        [ 0.06560652,  0.45328481, -0.30465294]]), array([[-1.20313147],
        [ 0.07101759],
        [ 0.31751729],
        ...,
        [-0.46815732],
        [-1.20313147],
        [-0.30553245]]))

In [10]:
def neural_net_model(X_data,input_dim):
    W_1 = tf.Variable(tf.random_uniform([input_dim,10]))
    b_1 = tf.Variable(tf.zeros([10]))
    layer_1 = tf.add(tf.matmul(X_data,W_1), b_1)
    layer_1 = tf.nn.tanh(layer_1)

    W_2 = tf.Variable(tf.random_uniform([10,10]))
    b_2 = tf.Variable(tf.zeros([10]))
    layer_2 = tf.add(tf.matmul(layer_1,W_2), b_2)
    layer_2 = tf.nn.tanh(layer_2)

    W_O = tf.Variable(tf.random_uniform([10,1]))
    b_O = tf.Variable(tf.zeros([1]))
    output = tf.add(tf.matmul(layer_2,W_O), b_O)

    return output,W_O

In [ ]:
import seaborn as sns
xs = tf.placeholder("float")
ys = tf.placeholder("float")

output,W_O = neural_net_model(xs,3)

cost = tf.reduce_mean(tf.square(output-ys))
train = tf.train.AdamOptimizer(0.01).minimize(cost)

correct_pred = tf.argmax(output, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

c_t = []
c_test = []


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver()
    y_t = Y_train
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    plt.xlabel('Days')
    plt.ylabel('Stock Value INR')
    plt.title('Nify Stock Index Prediction')
    ax.plot(range(len(Y_train)), y_t,label='Original')
    plt.ion()

    #saver.restore(sess,'yahoo_dataset.ckpt')
    for i in range(1000):
        sess.run([cost,train],feed_dict={xs:X_train, ys:Y_train})
#         for j in range(X_train.shape[0]):
#             sess.run([cost,train],feed_dict={xs:X_train[j,:].reshape(1,3), ys:Y_train[j]})

#         try:
#             ax.lines.remove(lines[0])
#         except Exception:
#             pass
        pred = sess.run(output, feed_dict={xs:X_train})
#         pred = denormalize(df_train,pred)
#         lines = ax.plot(range(len(y_train)), pred,'r-',label='Prediction')
#         plt.legend(loc='best')
#         plt.pause(0.1)

        c_t.append(sess.run(cost, feed_dict={xs:X_train,ys:Y_train}))
        c_test.append(sess.run(cost, feed_dict={xs:X_test,ys:Y_test}))
        if i%100==0:
            print('Epoch :',i,'Cost :',c_t[i])

    pred = sess.run(output, feed_dict={xs:X_test})
    
    for i in range(Y_test.shape[0]):
        print('Original :',Y_test[i],'Predicted :',pred[i])

    #plt.plot(range(50),c_t)
    #plt.plot(range(50),c_test)
    #plt.show()

    print('Cost :',sess.run(cost, feed_dict={xs:X_test,ys:Y_test}))
#     y_test = denormalize(df_test,y_test)
#     pred = denormalize(df_test,pred)
    plt.plot(range(Y_test.shape[0]),Y_test,label="Original Data")
    plt.plot(range(Y_test.shape[0]),pred,label="Predicted Data")
    plt.legend(loc='best')
    """plt.ylabel('Stock Value')
    plt.xlabel('Days')
    plt.title('Stock Market Nifty')"""
    #plt.show()
    if input('Save model ? [Y/N]') == 'Y':
        import os
        saver.save(sess, os.getcwd() + '/log_nn.ckpt')
        print('Model Saved')

Epoch : 0 Cost : 7.892241
Epoch : 100 Cost : 0.46813694
Epoch : 200 Cost : 0.39683208
Epoch : 300 Cost : 0.35665104
Epoch : 400 Cost : 0.32710218
Epoch : 500 Cost : 0.30741704
Epoch : 600 Cost : 0.29318792
Epoch : 700 Cost : 0.28286663
Epoch : 800 Cost : 0.2741602
Epoch : 900 Cost : 0.26512513
Original : [0.85830919] Predicted : [-1.1265969]
Original : [1.21445126] Predicted : [-1.1260741]
Original : [-1.20313147] Predicted : [-1.1198857]
Original : [1.10068818] Predicted : [-1.1208718]
Original : [1.33011821] Predicted : [-1.1209867]
Original : [0.316774] Predicted : [-1.1202204]
Original : [-0.03428237] Predicted : [-1.1204784]
Original : [-0.86206392] Predicted : [-1.1203706]
Original : [0.65327746] Predicted : [-1.1221685]
Original : [0.11158578] Predicted : [-1.1202576]
Original : [-0.97437953] Predicted : [-1.1205287]
Original : [0.53270738] Predicted : [-1.1214495]
Original : [-1.20443549] Predicted : [-1.1249366]
Original : [1.45336154] Predicted : [-1.1209617]
Original : [-1.2

Original : [-1.20313147] Predicted : [-1.1206667]
Original : [-1.20313147] Predicted : [-1.1223245]
Original : [-0.18627944] Predicted : [-1.1209283]
Original : [1.94260536] Predicted : [-1.1210155]
Original : [-1.20313147] Predicted : [-1.1207225]
Original : [0.25094686] Predicted : [-1.1205778]
Original : [1.77010903] Predicted : [-1.1209302]
Original : [-0.76550093] Predicted : [-1.1199989]
Original : [-0.33836778] Predicted : [-1.1203456]
Original : [-1.20313147] Predicted : [-1.1199012]
Original : [-1.20313147] Predicted : [-1.1203675]
Original : [-0.32604475] Predicted : [-1.120619]
Original : [-0.33204327] Predicted : [-1.1203651]
Original : [-1.20443549] Predicted : [-1.1223054]
Original : [-1.20443549] Predicted : [-1.1217673]
Original : [0.4997938] Predicted : [-1.1224716]
Original : [-0.32610996] Predicted : [-1.1223547]
Original : [0.27383248] Predicted : [-1.1258707]
Original : [-0.06243625] Predicted : [-1.1205132]
Original : [-0.75416896] Predicted : [-1.1202843]
Origina

In [ ]:
# Y_train,Y_pred
xid=0
yid=0
#     #Graphic display
# plt.plot(Xo_train[:,xid], Y_train[:,yid], 'r.', label='Original data')
# plt.plot(Xo_train[:,xid], Y_pred[:,yid],'b.', label='Fitted line')
plt.plot(Y_train[:,yid], Y_pred[:,yid],'b*', label='Fitted line')
plt.legend()
plt.show()

In [15]:
X_train.shape,X_train[j,:].reshape(1,3).shape
# X_train

((1124, 3), (1, 3))